In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

In [2]:
# Load the data and create engineered features
df = pd.read_csv('../../data/data.csv')
# Convert date columns to datetime
df[["observation_date", "date_of_introduction"]] = df[["observation_date", "date_of_introduction"]].apply(
    pd.to_datetime
)
# create number of days since introduction feature
df['days_since_introduction'] = (df['observation_date'] - df['date_of_introduction']).dt.days
df['initial_growth_potential'] = df['initial_female_count'] * df['leaf_area_cm2']

In [3]:
features = [
    'initial_female_count',
    'leaf_area_cm2',
    'population_density',
    'days_since_introduction',
    'initial_growth_potential'
]
target = 'cumulative_mite_count'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 1. Scale the Features (Crucial for ANNs) ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 2. Build the ANN Model Architecture ---
model = keras.Sequential([
    # Input layer and first hidden layer
    layers.Dense(64, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    # Second hidden layer
    layers.Dense(64, activation='relu'),
    # Output layer (1 neuron for the single output value)
    layers.Dense(1)
])

# --- 3. Compile the Model ---
model.compile(optimizer='adam',
              loss='mean_squared_error', # A common loss function for regression
              metrics=['mean_absolute_error'])

# --- 4. Train the Model ---
# We'll train for 100 epochs, meaning 100 passes over the training data
history = model.fit(
    X_train_scaled, y_train,
    epochs=100,
    validation_split=0.2, # Use part of the training data for validation
    verbose=0 # Suppress verbose output
)

# --- 5. Evaluate the Model ---
y_pred_ann = model.predict(X_test_scaled).flatten()
r2_ann = r2_score(y_test, y_pred_ann)
mae_ann = mean_absolute_error(y_test, y_pred_ann)
mean_squared_error_ann = mean_squared_error(y_test, y_pred_ann)
root_mean_squared_error_ann = root_mean_squared_error(y_test, y_pred_ann)

# --- Print the Results and Final Comparison ---
print("--- ANN Regressor Performance ---")
print(f"Mean Absolute Error (MAE): {mae_ann:.2f}\n")
print(f"Mean Squared Error (MSE): {mean_squared_error_ann:.2f}")
print(f"Root Mean Squared Error (RMSE): {root_mean_squared_error_ann:.2f}\n")
print(f"R-squared (R²): {r2_ann:.4f}")


e:\.UWU EDUCATION\spider reseach\code\.venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
--- ANN Regressor Performance ---
Mean Absolute Error (MAE): 35.47

Mean Squared Error (MSE): 3429.94
Root Mean Squared Error (RMSE): 58.57

R-squared (R²): 0.8271


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error

# Use the Scikeras wrapper to make Keras models compatible with scikit-learn
from scikeras.wrappers import KerasRegressor

# --- Assume 'df' is your DataFrame ---
# df = pd.read_csv('your_data.csv')

features = [
    'initial_female_count',
    'leaf_area_cm2',
    'population_density',
    'days_since_introduction',
    'initial_growth_potential'
]
target = 'cumulative_mite_count'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 1. Scale the Features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 2. Create a Function to Build the ANN Model ---
# This function is crucial for RandomizedSearchCV, as it allows us to create
# different model architectures by passing in hyperparameters as arguments.

def create_model(num_hidden_layers=1, 
                 neurons_per_layer=64, 
                 activation='relu', 
                 dropout_rate=0.2, 
                 learning_rate=0.001):
    """Dynamically creates a Keras Sequential model."""
    
    model = keras.Sequential()
    # The input shape is determined by the number of features
    model.add(layers.Input(shape=[X_train_scaled.shape[1]]))
    
    # Add the hidden layers in a loop
    for _ in range(num_hidden_layers):
        model.add(layers.Dense(neurons_per_layer, activation=activation))
        # Add Dropout for regularization to prevent overfitting
        model.add(layers.Dropout(dropout_rate))
        
    # Add the final output layer (1 neuron for a regression task)
    model.add(layers.Dense(1))
    
    # Compile the model with the specified learning rate
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'])
    return model

# --- 3. Define the Hyperparameter Search Space ---
# This dictionary tells RandomizedSearchCV which parameters to try and what values to sample from.
param_dist = {
    'model__num_hidden_layers': [1, 2, 3, 4],
    'model__neurons_per_layer': [32, 64, 128, 256],
    'model__activation': ['relu', 'tanh'],
    'model__dropout_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
    'model__learning_rate': [0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'epochs': [50, 100, 150]
}

# --- 4. Set Up Randomized Search CV ---
# Wrap the Keras model using the Scikeras wrapper
keras_regressor = KerasRegressor(model=create_model, verbose=0)

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=keras_regressor,
    param_distributions=param_dist,
    n_iter=25,  # Number of different parameter combinations to try
    cv=5,       # 5-fold cross-validation
    random_state=42,
    n_jobs=-1,  # Use all available CPU cores
    verbose=1   # Show progress
)

print("Starting hyperparameter search...")
# Fit the random search to the data
random_search.fit(X_train_scaled, y_train)

# --- 5. Analyze the Results ---
print("\nHyperparameter search complete.")
print(f"Best parameters found: {random_search.best_params_}")
print(f"Best cross-validation R² score: {random_search.best_score_:.4f}")

# --- 6. Evaluate the Best Model on the Test Set ---
# Get the best model found by the search
best_model = random_search.best_estimator_

# Make predictions on the unseen test data
y_pred_best_ann = best_model.predict(X_test_scaled)

# Calculate final performance metrics
r2_best_ann = r2_score(y_test, y_pred_best_ann)
mae_best_ann = mean_absolute_error(y_test, y_pred_best_ann)
mean_squared_error_best_ann = mean_squared_error(y_test, y_pred_best_ann)
root_mean_squared_error_best_ann = root_mean_squared_error(y_test, y_pred_best_ann)

# --- 7. Print the Final Performance ---
print("\n--- Tuned ANN Regressor Performance on Test Set ---")
print(f"R-squared (R²): {r2_best_ann:.4f}")
print(f"Mean Absolute Error (MAE): {mae_best_ann:.2f}")
print(f"Mean Squared Error (MSE): {mean_squared_error_best_ann:.2f}")
print(f"Root Mean Squared Error (RMSE): {root_mean_squared_error_best_ann:.2f}")

Starting hyperparameter search...
Fitting 5 folds for each of 25 candidates, totalling 125 fits

Hyperparameter search complete.
Best parameters found: {'model__num_hidden_layers': 4, 'model__neurons_per_layer': 256, 'model__learning_rate': 0.001, 'model__dropout_rate': 0.3, 'model__activation': 'relu', 'epochs': 100, 'batch_size': 32}
Best cross-validation R² score: 0.8232

--- Tuned ANN Regressor Performance on Test Set ---
R-squared (R²): 0.8348
Mean Absolute Error (MAE): 33.72


In [6]:
# Calculate final performance metrics
r2_best_ann = r2_score(y_test, y_pred_best_ann)
mae_best_ann = mean_absolute_error(y_test, y_pred_best_ann)
mean_squared_error_best_ann = mean_squared_error(y_test, y_pred_best_ann)
root_mean_squared_error_best_ann = root_mean_squared_error(y_test, y_pred_best_ann)

# --- 7. Print the Final Performance ---
print("\n--- Tuned ANN Regressor Performance on Test Set ---")
print(f"Mean Absolute Error (MAE): {mae_best_ann:.2f}")
print(f"Mean Squared Error (MSE): {mean_squared_error_best_ann:.2f}")
print(f"Root Mean Squared Error (RMSE): {root_mean_squared_error_best_ann:.2f}")
print(f"R-squared (R²): {r2_best_ann:.4f}")


--- Tuned ANN Regressor Performance on Test Set ---
Mean Absolute Error (MAE): 33.72
Mean Squared Error (MSE): 3276.16
Root Mean Squared Error (RMSE): 57.24
R-squared (R²): 0.8348
